In [9]:
def back_testing(final_data, buying_score = 0.3, selling_score = 0.7, print_type = True):
    
    try:

        # action index
        buying_index = [idx for idx, val in enumerate(final_data.FG_score) if val<=buying_score]
        selling_index = [idx for idx, val in enumerate(final_data.FG_score) if val>=selling_score]
        
        if len(buying_index)>0:
            buying_date = itemgetter(*buying_index)(final_data.datetime.reset_index(drop=True))
        else:
            buying_date = []
            
        if len(selling_index)>0:
            selling_date = itemgetter(*selling_index)(final_data.datetime.reset_index(drop=True))
        else:
            selling_date = []

        # per calculate
        per_list = [0,]
        for i in range(1,len(final_data)):
            plus_per = ((final_data.close.values[i]/final_data.close.values[i-1])-1) * 100
            per_list.append(plus_per)
        final_data["per"] = per_list

        asset_list = []
        asset = final_data['close'].values[0]
        stock = True

        start_list = []
        end_list = []
        # backtest action
        for i in final_data.index:

            # stock change
            if final_data.datetime[i] in buying_date and stock == False:
                stock = True # buying all stock
                end_list.append(final_data.datetime[i])
                if print_type == True: 
                    print("buying stock : " + str(final_data.datetime[i]))                    

            elif final_data.datetime[i] in selling_date and stock == True:
                stock = False # selling all stock
                start_list.append(final_data.datetime[i])
                if print_type == True:
                    print("sell stock : " + str(final_data.datetime[i]))
                    
            # asset calculation 
            if stock == True:
                asset = (final_data.per[i]/100  + 1) * asset
            asset_list.append(asset)

        if len(start_list) != len(end_list):
            end_list.append(final_data.datetime[i])

        anomaly_list = []
        for s,e in zip(start_list,end_list):
            anomaly_list.append([datetime.strptime(s,'%Y-%m-%d'), datetime.strptime(e,'%Y-%m-%d')])
            
        asset_list = list(np.round(asset_list,2))
            
        return asset_list, anomaly_list
    
    except Exception as ex:
        return final_data.close, []


In [2]:
# # make candid list

# threshold_list = []

# for b in np.arange (0.1,1,0.1):
#     for s in np.arange(0.1,1,0.1):
#         threshold_list.append([b,s])

In [ ]:
# best_close = 0
# best_parameter = [0,0] 
# for i in threshold_list:
    
#     tmp = back_testing(final_data, buying_score = i[0], selling_score = i[1])
    
#     if best_close < tmp[len(tmp)-1]:
#         print(i)
#         print("as-is : " + str(best_close) + " / to-be : " + str(tmp[len(tmp)-1]))
#         best_close = tmp[len(tmp)-1]
#         best_parameter = i
